## Test Experiments

In [1]:
from utils.utils import *
import optuna
from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend
from experiments.experiments_gkan import ExperimentsGKAN
from experiments.experiments_mpnn import ExperimentsMPNN

storage = JournalStorage(JournalFileBackend("optuna_journal_storage.log"))

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Kuramoto MPNN

In [2]:
optuna.delete_study(study_name="model-kuramoto-mpnn-test-new-dataset", storage=storage)

In [3]:
config_path = './configs/config_kuramoto_mpnn.yml'
config = load_config(config_path)
config['epochs'] = 10

exp = ExperimentsMPNN(
    config=config,
    n_trials=1,
    study_name='test-new-dataset',
    process_id=0,
)

Processing...


Building the dataset...


Done!


In [4]:
exp.run()

[I 2025-05-09 17:17:47,063] A new study created in Journal with name: model-kuramoto-mpnn-test-new-dataset
[I 2025-05-09 17:18:24,861] Trial 0 finished with value: 0.016433067619800568 and parameters: {'lr': 0.01735137743570187, 'lamb': 0.0, 'batch_size': 32, 'n_hidden_layers_g_net': 2, 'hidden_dims_g_net': 48, 'af_g_net': 'tanh', 'drop_p_g_net': 0.04457723703003285, 'n_hidden_layers_h_net': 1, 'hidden_dims_h_net': 32, 'af_h_net': 'relu', 'drop_p_h_net': 0.12874245837341808}. Best is trial 0 with value: 0.016433067619800568.


### Epidemics MPNN

In [8]:
optuna.delete_study(study_name="model-epidemics-mpnn-test-epidemics", storage=storage)

In [9]:
config_path = './configs/config_epidemics_mpnn.yml'
config = load_config(config_path)
config['epochs'] = 10

exp = ExperimentsMPNN(
    config=config,
    n_trials=1,
    study_name='test-epidemics',
    process_id=0,
)

Processing...


Building the dataset...


Done!


In [10]:
exp.run()

[I 2025-05-09 17:06:59,961] A new study created in Journal with name: model-epidemics-mpnn-test-epidemics


Epoch: 0 	 Training loss: 0.01639 	 Val Loss: 0.01206 	 Tot Loss: 0.01639
Epoch: 1 	 Training loss: 0.00991 	 Val Loss: 0.01101 	 Tot Loss: 0.00991
Epoch: 2 	 Training loss: 0.00811 	 Val Loss: 0.00909 	 Tot Loss: 0.00811
Epoch: 3 	 Training loss: 0.00670 	 Val Loss: 0.00712 	 Tot Loss: 0.00670
Epoch: 4 	 Training loss: 0.00538 	 Val Loss: 0.00533 	 Tot Loss: 0.00538
Epoch: 5 	 Training loss: 0.00387 	 Val Loss: 0.00223 	 Tot Loss: 0.00387
Epoch: 6 	 Training loss: 0.00268 	 Val Loss: 0.00225 	 Tot Loss: 0.00268
Epoch: 7 	 Training loss: 0.00206 	 Val Loss: 0.00136 	 Tot Loss: 0.00206
Epoch: 8 	 Training loss: 0.00183 	 Val Loss: 0.00119 	 Tot Loss: 0.00183
Epoch: 9 	 Training loss: 0.00148 	 Val Loss: 0.00113 	 Tot Loss: 0.00148

Loading best model found at epoch 9 with val loss 0.0011312818387523293


[I 2025-05-09 17:07:33,036] Trial 0 finished with value: 0.0011312818387523293 and parameters: {'lr': 0.0030889538851373733, 'lamb': 0.0, 'batch_size': 16, 'n_hidden_layers_g_net': 1, 'hidden_dims_g_net': 64, 'af_g_net': 'tanh', 'drop_p_g_net': 0.00016167883486477297, 'n_hidden_layers_h_net': 1, 'hidden_dims_h_net': 64, 'af_h_net': 'relu', 'drop_p_h_net': 0.00037596544202597135}. Best is trial 0 with value: 0.0011312818387523293.


Test loss: 0.0006704181432723999


In [16]:
model_path_mpnn = './saved_models_optuna/model-epidemics-mpnn/test-epidemics/0/mpnn'

pysr_model = lambda : get_pysr_model(
    model_selection="score",
    n_iterations=150
)

mpnn_symb = fit_mpnn(
    model_path=model_path_mpnn,
    pysr_model=pysr_model,
    sample_size=9000,
    message_passing=False
)

/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


In [17]:
mpnn_symb

\sum_{j}( 0.19483042*exp(-exp(x_i))) - log(log(0.1571578*x_i + 1) + 1) + 0.21526255

### Neuronal MPNN

In [11]:
optuna.delete_study(study_name="model-neuronal-mpnn-test-neuronal", storage=storage)

In [12]:
config_path = './configs/config_neuronal_mpnn.yml'
config = load_config(config_path)
config['epochs'] = 10

exp = ExperimentsMPNN(
    config=config,
    n_trials=1,
    study_name='test-neuronal',
    process_id=0,
)

Processing...


Building the dataset...


Done!


In [13]:
exp.run()

[I 2025-05-09 17:07:45,709] A new study created in Journal with name: model-neuronal-mpnn-test-neuronal


Epoch: 0 	 Training loss: 0.15600 	 Val Loss: 0.09005 	 Tot Loss: 0.15600
Epoch: 1 	 Training loss: 0.05407 	 Val Loss: 0.07933 	 Tot Loss: 0.05407
Epoch: 2 	 Training loss: 0.05492 	 Val Loss: 0.00986 	 Tot Loss: 0.05492
Epoch: 3 	 Training loss: 0.02307 	 Val Loss: 0.02946 	 Tot Loss: 0.02307
Epoch: 4 	 Training loss: 0.02772 	 Val Loss: 0.00445 	 Tot Loss: 0.02772
Epoch: 5 	 Training loss: 0.01449 	 Val Loss: 0.02453 	 Tot Loss: 0.01449
Epoch: 6 	 Training loss: 0.01730 	 Val Loss: 0.01004 	 Tot Loss: 0.01730
Epoch: 7 	 Training loss: 0.00799 	 Val Loss: 0.00930 	 Tot Loss: 0.00799
Epoch: 8 	 Training loss: 0.01102 	 Val Loss: 0.00358 	 Tot Loss: 0.01102
Epoch: 9 	 Training loss: 0.00579 	 Val Loss: 0.01125 	 Tot Loss: 0.00579

Loading best model found at epoch 8 with val loss 0.003580222139135003
Test loss: 0.004640685394406319


[I 2025-05-09 17:08:23,447] Trial 0 finished with value: 0.003580222139135003 and parameters: {'lr': 0.022198182869050127, 'lamb': 0.0, 'batch_size': 32, 'n_hidden_layers_g_net': 2, 'hidden_dims_g_net': 32, 'af_g_net': 'softplus', 'drop_p_g_net': 0.0005185973727913854, 'n_hidden_layers_h_net': 1, 'hidden_dims_h_net': 40, 'af_h_net': 'tanh', 'drop_p_h_net': 0.018136511652483187}. Best is trial 0 with value: 0.003580222139135003.


In [5]:
model_path_mpnn = './saved_models_optuna/model-neuronal-mpnn/test-neuronal/0/mpnn'

pysr_model = lambda : get_pysr_model(
    model_selection="score",
    n_iterations=150
)

mpnn_symb = fit_mpnn(
    model_path=model_path_mpnn,
    pysr_model=pysr_model,
    sample_size=9000,
    message_passing=False
)

/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


In [6]:
mpnn_symb


\sum_{j}( log(log(tanh(x_j/x_i) + 1) + 1)) + 0.14861982/tanh(exp(x_i))

In [18]:
config_path = './configs/config_neuronal.yml'
config = load_config(config_path)
config['epochs'] = 5

exp = ExperimentsGKAN(
    config=config,
    n_trials=1,
    study_name='test-neuronal',
    process_id=0,
)

In [19]:
exp.run()

[I 2025-05-09 14:21:16,405] Using an existing study with name 'model-neuronal-gkan-test-neuronal' instead of creating a new one.


Epoch: 0 	 Training loss: 0.00051 	 Val Loss: 0.00028 	 Tot Loss: 0.00543
Epoch: 1 	 Training loss: 0.00028 	 Val Loss: 0.00014 	 Tot Loss: 0.00507
Epoch: 2 	 Training loss: 0.00016 	 Val Loss: 0.00007 	 Tot Loss: 0.00483
Epoch: 3 	 Training loss: 0.00012 	 Val Loss: 0.00005 	 Tot Loss: 0.00453
Epoch: 4 	 Training loss: 0.00011 	 Val Loss: 0.00004 	 Tot Loss: 0.00443

Loading best model found at epoch 4 with val loss 4.3011452362407e-05
Test loss: 0.000152996537508443


[I 2025-05-09 14:24:39,155] Trial 1 finished with value: 4.3011452362407e-05 and parameters: {'lr': 0.014578887605134958, 'lamb': 1.0, 'batch_size': 32, 'use_orig_reg': True, 'lamb_g_net': 9.722521843140844e-05, 'lamb_h_net': 0.0017345619188380956, 'grid_size_g_net': 9, 'spline_order_g_net': 3, 'range_limit_g_net': 1, 'mu_1_g_net': 0.6, 'mu_2_g_net': 1.0, 'hidden_dim_g_net': 1, 'grid_size_h_net': 7, 'spline_order_h_net': 2, 'range_limit_h_net': 3, 'mu_1_h_net': 1.0, 'mu_2_h_net': 0.7, 'hidden_dim_h_net': 2}. Best is trial 1 with value: 4.3011452362407e-05.


### Biochemical MPNN

In [14]:
optuna.delete_study(study_name="model-biochemical-mpnn-test-biochemical", storage=storage)

In [15]:
config_path = './configs/config_biochemical_mpnn.yml'
config = load_config(config_path)
config['epochs'] = 10

exp = ExperimentsMPNN(
    config=config,
    n_trials=1,
    study_name='test-biochemical',
    process_id=0,
)

Processing...


Building the dataset...


Done!


In [16]:
exp.run()

[I 2025-05-09 17:08:40,883] A new study created in Journal with name: model-biochemical-mpnn-test-biochemical


Epoch: 0 	 Training loss: 0.01061 	 Val Loss: 0.00656 	 Tot Loss: 0.01061
Epoch: 1 	 Training loss: 0.00874 	 Val Loss: 0.00552 	 Tot Loss: 0.00874
Epoch: 2 	 Training loss: 0.00779 	 Val Loss: 0.00526 	 Tot Loss: 0.00779
Epoch: 3 	 Training loss: 0.00718 	 Val Loss: 0.00465 	 Tot Loss: 0.00718
Epoch: 4 	 Training loss: 0.00654 	 Val Loss: 0.00416 	 Tot Loss: 0.00654
Epoch: 5 	 Training loss: 0.00580 	 Val Loss: 0.00388 	 Tot Loss: 0.00580
Epoch: 6 	 Training loss: 0.00511 	 Val Loss: 0.00322 	 Tot Loss: 0.00511
Epoch: 7 	 Training loss: 0.00457 	 Val Loss: 0.00279 	 Tot Loss: 0.00457
Epoch: 8 	 Training loss: 0.00387 	 Val Loss: 0.00227 	 Tot Loss: 0.00387
Epoch: 9 	 Training loss: 0.00335 	 Val Loss: 0.00189 	 Tot Loss: 0.00335

Loading best model found at epoch 9 with val loss 0.001890188199467957
Test loss: 0.0018362425034865737


[I 2025-05-09 17:09:18,946] Trial 0 finished with value: 0.001890188199467957 and parameters: {'lr': 0.0011331826087084096, 'lamb': 0.0, 'batch_size': 16, 'n_hidden_layers_g_net': 2, 'hidden_dims_g_net': 24, 'af_g_net': 'tanh', 'drop_p_g_net': 0.012105133221616455, 'n_hidden_layers_h_net': 1, 'hidden_dims_h_net': 16, 'af_h_net': 'softplus', 'drop_p_h_net': 0.01289533029330185}. Best is trial 0 with value: 0.001890188199467957.


In [11]:
model_path_mpnn = './saved_models_optuna/model-biochemical-mpnn/test-biochemical/0/mpnn'

pysr_model = lambda : get_pysr_model(
    model_selection="score",
    n_iterations=200
)

mpnn_symb = fit_mpnn(
    model_path=model_path_mpnn,
    pysr_model=pysr_model,
    sample_size=9000,
    message_passing=False
)

/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


In [12]:
mpnn_symb

\sum_{j}( -0.15344319*x_i*exp(x_j)) - 0.14971581*x_i + 0.87793374

In [23]:
config_path = './configs/config_biochemical.yml'
config = load_config(config_path)
config['epochs'] = 5

exp = ExperimentsGKAN(
    config=config,
    n_trials=1,
    study_name='test-biochemical',
    process_id=0,
)

In [24]:
exp.run()

[I 2025-05-09 14:43:07,097] A new study created in Journal with name: model-biochemical-gkan-test-biochemical


Epoch: 0 	 Training loss: 0.00011 	 Val Loss: 0.00007 	 Tot Loss: 0.00409
Epoch: 1 	 Training loss: 0.00010 	 Val Loss: 0.00006 	 Tot Loss: 0.00416
Epoch: 2 	 Training loss: 0.00010 	 Val Loss: 0.00005 	 Tot Loss: 0.00426
Epoch: 3 	 Training loss: 0.00009 	 Val Loss: 0.00005 	 Tot Loss: 0.00439
Epoch: 4 	 Training loss: 0.00009 	 Val Loss: 0.00004 	 Tot Loss: 0.00456

Loading best model found at epoch 4 with val loss 4.181330223218538e-05
Test loss: 2.1962674509268254e-05


[I 2025-05-09 14:45:45,104] Trial 0 finished with value: 4.181330223218538e-05 and parameters: {'lr': 0.0029212460068246317, 'lamb': 1.0, 'batch_size': 16, 'use_orig_reg': True, 'lamb_g_net': 0.0012543290658560168, 'lamb_h_net': 1.2825379943612229e-06, 'grid_size_g_net': 8, 'spline_order_g_net': 1, 'range_limit_g_net': 1, 'mu_1_g_net': 0.7, 'mu_2_g_net': 0.9, 'hidden_dim_g_net': 3, 'grid_size_h_net': 6, 'spline_order_h_net': 2, 'range_limit_h_net': 3, 'mu_1_h_net': 0.6, 'mu_2_h_net': 0.5, 'hidden_dim_h_net': 3}. Best is trial 0 with value: 4.181330223218538e-05.
